In [15]:
!pip install smolagents python-dotenv sqlalchemy --upgrade -q


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from google.colab import userdata

my_token = userdata.get('HF_TOKEN')
with open('.env', 'w') as f:
    f.write(f"HF_TOKEN={my_token}")


In [17]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [18]:
db = "world"
path_json = f"dataset/{db}/{db}.json"
path_sql = f"dataset/{db}/{db}.sqlite"

In [19]:
import json

question = None
with open(path_json, "r") as f:
    questions = json.load(f)

with open("golds.json", 'r') as v:
  golds = json.load(v)
  q_ids = {g["question_id"]: g for g in golds}

In [20]:
from sqlalchemy import create_engine, inspect, text



db_path = f"sqlite:///{path_sql}"
db_name = f"{db}.sqlite"

if not os.path.exists(path_sql):
    print("WARNING: db not found.")

engine = create_engine(db_path)

In [21]:
inspector = inspect(engine)
table_names = inspector.get_table_names()

schema = "Database Schema:\n"

for table in table_names:
    schema += f"Table: {table}\n"
    columns = inspector.get_columns(table)
    for col in columns:
        schema += f"  - {col['name']} ({col['type']})\n"

print(schema)

Database Schema:
Table: City
  - ID (INTEGER)
  - Name (TEXT)
  - CountryCode (TEXT)
  - District (TEXT)
  - Population (INTEGER)
Table: Country
  - Code (TEXT)
  - Name (TEXT)
  - Continent (TEXT)
  - Region (TEXT)
  - SurfaceArea (REAL)
  - IndepYear (INTEGER)
  - Population (INTEGER)
  - LifeExpectancy (REAL)
  - GNP (REAL)
  - GNPOld (REAL)
  - LocalName (TEXT)
  - GovernmentForm (TEXT)
  - HeadOfState (TEXT)
  - Capital (INTEGER)
  - Code2 (TEXT)
Table: CountryLanguage
  - CountryCode (TEXT)
  - Language (TEXT)
  - IsOfficial (TEXT)
  - Percentage (REAL)



In [22]:
from smolagents import tool
@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.

    Args:
        query: The query to perform.
    """
    output = ""
    MAX_ROWS = 10
    with engine.connect() as con:
        rows = con.execute(text(query))
        for i, row in enumerate(rows):
          if i > MAX_ROWS:
            output += "\n...Output truncated."
            return output
          else:
            output += "\n" + str(row)

    return output

In [ ]:
from smolagents import CodeAgent, InferenceClientModel, EMPTY_PROMPT_TEMPLATES

# Define the optimized system prompt combining PDF Examples + Smolagents Technical Logic
system_prompt = """
You are an expert Data Scientist specialized in Text-to-SQL tasks. Your goal is to answer natural language questions by generating valid, executable SQL queries.

You will be given a task to solve as best you can.
The tools at your disposal are:
- sql_engine(query: str) -> str: Executes the provided SQL query and returns the results as a string.
- final_answer(sql_string: str) -> None: Finalizes the interaction by returning the SQL query string as the answer.
The tools must be called within the {{code_block_opening_tag}}...{{code_block_closing_tag}} tags.

You may run exploratory or validation queries using the sql_engine function to validate your assumptions before producing the final SQL query.

PROTOCOL:
1. Reasoning Trace: Explicitly state your plan in the 'Thought' section.
2. Schema Understanding: Use the provided database schema to understand:
   - table names
   - column names
   - primary/foreign key relationships
   - bridge tables
3. After every exploratory or validation query executed through sql_engine, you MUST:
   - Read the result.
   - Produce a new explicit `Thought:` section interpreting the output.
   - Decide the next step based on that interpretation before running another code block.

CRITICAL RULES (YOU MUST FOLLOW):
1. Python Syntax Only: The code block contains PYTHON code.
2. SQL as Strings: Always pass SQL as a string to `sql_engine`: `print(sql_engine(query="SELECT ..."))`
3. NO PYTHON DATA MANIPULATION: Do not fetch data into Python to filter it. Write a single SQL query using JOINs or Subqueries.
4. FINAL ANSWER FORMAT: NO BLIND SUBMISSIONS:
   - Your final goal is the SQL QUERY STRING, not the data result.
   - DO NOT EXECUTE the final query inside `final_answer`:
    * WRONG: `final_answer(sql_engine(query="SELECT..."))` <-- returns data (BAD)
    * CORRECT: `final_answer("SELECT...")` <-- returns SQL (GOOD)

Example 1
Question:
"Return the names of companies in the ‘AI’ sector that were founded before 2015 and have at least two offices in France."

Thought: I need to find companies meeting three criteria: sector 'AI', founded before 2015, and having > 1 office in France.
The schema involves 'Company' and 'Office' tables, likely joined by 'cid'.
Since this involves a JOIN, a string filter ('AI'), and a GROUP BY condition, I will run a validation query to verify the logic before finalizing.

{{code_block_opening_tag}}
validation_query = \"\"\"
SELECT c.cid, c.name, COUNT(*) AS fr_offices
FROM Company c
JOIN Office o ON o.cid=c.cid
WHERE c.sector='AI' AND c.founded_year<2015 AND o.country='France'
GROUP BY c.cid, c.name
HAVING COUNT(*)>=2
LIMIT 3
\"\"\"
print(sql_engine(query=validation_query))
{{code_block_closing_tag}}

[ANS]
cid | name | fr_offices
2 | Acme AI | 2
[/ANS]

Thought: The test query returned a valid result, confirming the JOIN condition and filters work as expected. I can now generate the final SQL query string.

{{code_block_opening_tag}}
final_sql = "\"\"
SELECT c.name
FROM Company AS c
JOIN Office AS o ON o.cid = c.cid
WHERE c.sector = 'AI'
AND c.founded_year < 2015
AND o.country = 'France'
GROUP BY c.cid, c.name
HAVING COUNT(*) >= 2
"\"\"
final_answer(final_sql)
{{code_block_closing_tag}}

Example 2
Question:
"How many scenes are there in Act 1 in Twelfth Night?

Thought: I need to count the number of scenes in Act 1 for the work 'Twelfth Night'.
I can identify the tables 'works' and 'chapters' from the schema.
- 'works' contains the title.
- 'chapters' contains Act and Scene information.
- They are linked by 'work_id'.
This is a standard JOIN and COUNT operation. I am confident in the schema structure, so I will write the final query directly.

{{code_block_opening_tag}}
final_sql = "\"\"
SELECT COUNT(*) as n_Scenes
FROM works w JOIN chapters c ON c.work_id = w.id
WHERE w.Title = 'Twelfth Night' AND c.Act = 1
"\"\"
final_answer(final_sql)
{{code_block_closing_tag}}

────────────────────────────

Here are the rules you should always follow to solve your task:
1. Always provide a 'Thought:' sequence, and a '{{code_block_opening_tag}}' sequence ending with '{{code_block_closing_tag}}', else you will fail.
2. Use only variables that you have defined!
3. Always use the right arguments for the tools. DO NOT pass the arguments as a dict as in 'answer = wikipedia_search({'query': \"What is the place where James Bond lives?\"})', but use the arguments directly as in 'answer = wikipedia_search(query=\"What is the place where James Bond lives?\")'.
4. Don't name any new variable with the same name as a tool: for instance don't name a variable 'final_answer'.
5. Never create any notional variables in our code, as having these in your logs will derail you from the true variables.
6. You can use imports in your code, but only from the following list of modules: {{authorized_imports}}
7. The state persists between code executions: so if in one step you've created variables or imported modules, these will all persist.
8. Don't give up! You're in charge of solving the task, not providing directions to solve it.
"""


my_templates_dict = EMPTY_PROMPT_TEMPLATES.copy()
my_templates_dict["system_prompt"] = system_prompt

In [24]:
agent = CodeAgent(
    tools=[sql_engine],
    model=InferenceClientModel(model_id="Qwen/Qwen3-8B", token=my_token),
    prompt_templates=my_templates_dict,
    verbosity_level=2
)

In [25]:
import re

def get_stats(agent):


    log_parts = []
    sql_query = None
    reasoning_len = 0
    tool_call_count = 0
    errors_count = 0
    is_final_answer = False

    for i, step in enumerate(agent.memory.steps):

        if i == 0: continue # Skip the task step

        if step.is_final_answer:
          sql_query = step.action_output.strip()
          is_final_answer = True

        thought = getattr(step, 'model_output', getattr(step, 'thought', None))
        if thought:
            print(thought)
            clean_thought = re.sub(r'<code>.*?</code>', '', thought, flags=re.DOTALL)
            clean_thought = clean_thought.replace('\n', ' ').strip()
            if clean_thought:
                log_parts.append(f"{clean_thought}")

        # call
        if hasattr(step, 'tool_calls') and step.tool_calls and not is_final_answer:
            for tool_call in step.tool_calls:
              if step.is_final_answer:
                continue
              tool_call_count += 1
              args = getattr(tool_call, 'arguments', str(tool_call))

              match_triple = re.search(r'=\s*"""(.*?)"""', args, re.DOTALL)
              match_direct = re.search(r'sql_engine\s*\(\s*query\s*=\s*["\'](.*?)["\']', args, re.DOTALL)

              found_sql = None
              if match_triple:
                  found_sql = match_triple.group(1)
              elif match_direct:
                  found_sql = match_direct.group(1)

              if found_sql:
                  # Flatten SQL (remove newlines for single-line log)
                  flat_sql = found_sql.replace('\n', ' ').replace('\\n', ' ').replace("   ", " ").strip()
                  log_parts.append(f"[CALL] {flat_sql}")

        # ans - obs
        if hasattr(step, 'observations') and step.observations and not is_final_answer:

          obs = str(step.observations).strip()


          obs = obs.replace("Execution logs:", "").replace("Last output from code snippet:", "")
          obs = re.sub(r'\bNone\b', '', obs)
          obs = re.sub(r"^\('(.+)',\)$", r"\1", obs.strip(), flags=re.MULTILINE)
          obs = re.sub(r"^\('(.+)'\)$", r"\1", obs.strip(), flags=re.MULTILINE)

          obs_clean = obs.strip().replace('\n', ' ')

          if not obs_clean or re.fullmatch(r'[\[\]\(\)\s,]*', obs_clean):
              log_parts.append("[ANS] (no rows) [/ANS]")

          # Truncate if too long
          if len(obs_clean) > 200:
              obs_clean = obs_clean[:200] + "... [truncated]"

          if obs_clean:
              log_parts.append(f"[ANS] {obs_clean} [/ANS]")

        # errors
        if hasattr(step, 'error') and step.error:
             err_clean = str(step.error).replace('\n', ' ')
             errors_count += 1
             log_parts.append(f"[ERROR] {err_clean}")


    # Join everything with a single newline between steps
    full_log_string = " ".join(log_parts)
    full_log_string = full_log_string.replace("\\", "").replace("\"", "\'")
    reasoning_len = len(full_log_string)
    sql_query = sql_query.replace("\\n",  "").replace("\n", " ").replace("\\'", "'").strip()


    return full_log_string, sql_query, tool_call_count, errors_count, reasoning_len


In [ ]:
def compute_execution_accuracy(gt_results, predict_results):
  num_correct = 0
  num_queries = len(gt_results)
  mismatch_idx = []

  for i, result in enumerate(gt_results):
      if set(result['results']) == set(predict_results[i]['results']):
          num_correct += 1
      else:
          mismatch_idx.append(i)

  acc = (num_correct / num_queries)

  return acc

In [27]:
import sqlite3
def run_query(db_path, query):
  conn = sqlite3.connect(db_path)
  try:
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    conn.close()

    # Flatten results and convert to list of strings
    return [row[0] for row in rows], True
  except:
    return [], False

In [28]:
import time

traces = []
i = 1
for q in questions:
  trace_accuracy = None
  question = q["questions"]
  evidence = q["evidence"]
  difficulty = q["difficulty"]
  q_id = q["question_id"]

  gt_query = q_ids[q_id]["target_sql"]

  USER_PROMPT = f"""DB Schema: {schema}. Question: {evidence}. {question}"""

  print(f"--- Question {i} ---")
  i += 1

  start_time = time.time()
  agent.run(USER_PROMPT)
  end_time = time.time()

  log_string, pred_query, tool_call_count, errors_count, reasoning_len = get_stats(agent)

  rows_gt, _ = run_query(path_sql, gt_query)
  gt_res = [{"results": rows_gt}]

  rows_pred, is_valid_sql = run_query(path_sql, pred_query)
  pred_res= [{"results": rows_pred}]

  if is_valid_sql:
    acc = compute_execution_accuracy(gt_res, pred_res)

  else:
    trace_accuracy = 0
    acc = 0


  complete_trace = {
    "question_id": q_id,
    "input": USER_PROMPT,
    "output": log_string,
    "difficulty": difficulty,
    "pred_query": pred_query,
    "target_query": gt_query,
    "tool_call_count": tool_call_count,
    "error_count": errors_count,
    "latency": round(end_time-start_time, 2),
    "reasoning_len": reasoning_len,
    "execution_accuracy": int(acc),
    "trace_accuracy": trace_accuracy
  }

  traces.append(complete_trace)


with open(f"traces_CA_nf/{db}_nf_traces.json", "w") as f:
  json.dump(traces, f, indent=2, ensure_ascii=False)

master_file_path = "traces_CA_nf/traces_CA_nf.json"
master_traces = []

with open(master_file_path, "r") as f:
  file_content = f.read()
  if file_content:
    master_traces = json.loads(file_content)

master_traces.extend(traces)

with open(master_file_path, "w") as f:
    json.dump(master_traces, f, indent=2, ensure_ascii=False)

--- Question 1 ---


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ DB Schema: Database Schema:                                                                                     │
│ Table: City                                                                                                     │
│   - ID (INTEGER)                                                                                                │
│   - Name (TEXT)                                                                                                 │
│   - CountryCode (TEXT)                                                                                          │
│   - District (TEXT)                                                                                             │
│   - Population (INTEGER)                                                                                        │
│ Table: Country                                                                                                  │
│   - Code (TEXT)                                                                                                 │
│   - Name (TEXT)                                                                                                 │
│   - Continent (TEXT)                                                                                            │
│   - Region (TEXT)                                                                                               │
│   - SurfaceArea (REAL)                                                                                          │
│   - IndepYear (INTEGER)                                                                                         │
│   - Population (INTEGER)                                                                                        │
│   - LifeExpectancy (REAL)                                                                                       │
│   - GNP (REAL)                                                                                                  │
│   - GNPOld (REAL)                                                                                               │
│   - LocalName (TEXT)                                                                                            │
│   - GovernmentForm (TEXT)                                                                                       │
│   - HeadOfState (TEXT)                                                                                          │
│   - Capital (INTEGER)                                                                                           │
│   - Code2 (TEXT)                                                                                                │
│ Table: CountryLanguage                                                                                          │
│   - CountryCode (TEXT)                                                                                          │
│   - Language (TEXT)                                                                                             │
│   - IsOfficial (TEXT)                                                                                           │
│   - Percentage (REAL)                                                                                           │
│ . Question: . What are the cities for country called "Â´Uman" in local name.                                    │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-8B ──────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                   
                                                                                                                   
Thought: I need to find cities belonging to the country with LocalName "Â´Uman". This requires joining the City and
Country tables on CountryCode, then filtering by LocalName.                                                        
                                                                                                                   
<code>                                                                                                             
validation_query = """                                                                                             
SELECT c.Name                                                                                                      
FROM City AS c                                                                                                     
JOIN Country AS co ON c.CountryCode = co.Code                                                                      
WHERE co.LocalName = 'Â´Uman'                                                                                      
LIMIT 3                                                                                                            
"""                                                                                                                
print(sql_engine(query=validation_query))                                                                          
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  validation_query = """                                                                                           
  SELECT c.Name                                                                                                    
  FROM City AS c                                                                                                   
  JOIN Country AS co ON c.CountryCode = co.Code                                                                    
  WHERE co.LocalName = 'Â´Uman'                                                                                    
  LIMIT 3                                                                                                          
  """                                                                                                              
  print(sql_engine(query=validation_query))                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                   
                                                                                                                   
Thought: The validation query successfully retrieved districts associated with the country having the maximum      
SurfaceArea. The use of DISTINCT ensures unique district names, and the subquery correctly identifies the largest  
surface area. The LIMIT 3 was only for testing; the final query should include all districts without restriction.  
                                                                                                                   
<code>                                                                                                             
final_sql = """                                                                                                    
SELECT DISTINCT c.District                                                                                         
FROM City c                                                                                                        
JOIN Country co ON c.CountryCode = co.Code                                                                         
WHERE co.SurfaceArea = (                                                                                           
    SELECT MAX(SurfaceArea)                                                                                        
    FROM Country                                                                                                   
)                                                                                                                  
"""                                                                                                                
final_answer(final_sql)                                                                                            
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_sql = """                                                                                                  
  SELECT DISTINCT c.District                                                                                       
  FROM City c                                                                                                      
  JOIN Country co ON c.CountryCode = co.Code                                                                       
  WHERE co.SurfaceArea = (                                                                                         
      SELECT MAX(SurfaceArea)                                                                                      
      FROM Country                                                                                                 
  )                                                                                                                
  """                                                                                                              
  final_answer(final_sql)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 
SELECT DISTINCT c.District
FROM City c
JOIN Country co ON c.CountryCode = co.Code
WHERE co.SurfaceArea = (
    SELECT MAX(SurfaceArea)
    FROM Country
)

[Step 2: Duration 16.02 seconds| Input tokens: 3,570 | Output tokens: 2,139]



Thought: The goal is to find the districts in the City table that belong to the country with the largest SurfaceArea. First, I need to identify the country with the maximum SurfaceArea from the Country table. Then, I will retrieve the distinct districts from the City table where the CountryCode matches that country's Code. This requires a subquery to find the max SurfaceArea and link it to the corresponding country's Code.

<code>
validation_query = """
SELECT DISTINCT c.District
FROM City c
JOIN Country co ON c.CountryCode = co.Code
WHERE co.SurfaceArea = (
    SELECT MAX(SurfaceArea)
    FROM Country
)
LIMIT 3
"""
print(sql_engine(query=validation_query))
</code>


Thought: The validation query successfully retrieved districts associated with the country having the maximum SurfaceArea. The use of DISTINCT ensures unique district names, and the subquery correctly identifies the largest surface area. The LIMIT 3 was only for testing; the final query should include all districts witho